In [201]:
from __future__ import division

import os.path
import pandas as pd
import numpy as np
import csv
import glob

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import uuid

%matplotlib inline


In [242]:
header = ['SPID','SA','UOM','DIR','DATE','RS','NAICS','APCT',
 '0:00',
 '0:15',
 '0:30',
 '0:45',
 '1:00',
 '1:15',
 '1:30',
 '1:45',
 '2:00',
 '2:15',
 '2:30',
 '2:45',
 '3:00',
 '3:15',
 '3:30',
 '3:45',
 '4:00',
 '4:15',
 '4:30',
 '4:45',
 '5:00',
 '5:15',
 '5:30',
 '5:45',
 '6:00',
 '6:15',
 '6:30',
 '6:45',
 '7:00',
 '7:15',
 '7:30',
 '7:45',
 '8:00',
 '8:15',
 '8:30',
 '8:45',
 '9:00',
 '9:15',
 '9:30',
 '9:45',
 '10:00',
 '10:15',
 '10:30',
 '10:45',
 '11:00',
 '11:15',
 '11:30',
 '11:45',
 '12:00',
 '12:15',
 '12:30',
 '12:45',
 '13:00',
 '13:15',
 '13:30',
 '13:45',
 '14:00',
 '14:15',
 '14:30',
 '14:45',
 '15:00',
 '15:15',
 '15:30',
 '15:45',
 '16:00',
 '16:15',
 '16:30',
 '16:45',
 '17:00',
 '17:15',
 '17:30',
 '17:45',
 '18:00',
 '18:15',
 '18:30',
 '18:45',
 '19:00',
 '19:15',
 '19:30',
 '19:45',
 '20:00',
 '20:15',
 '20:30',
 '20:45',
 '21:00',
 '21:15',
 '21:30',
 '21:45',
 '22:00',
 '22:15',
 '22:30',
 '22:45',
 '23:00',
 '23:15',
 '23:30',
 '23:45',
 'ZIPCODE',
 'SUBLAP']

In [203]:
df = pd.read_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/95202.csv',header=None)
df.drop(df.columns[[0]], inplace=True, axis=1)
df.columns = header

df.DATE = pd.to_datetime(df.DATE, format='%m/%d/%Y')
df.index = pd.to_datetime(df.DATE, format='%m/%d/%Y')

In [204]:
customers = pd.read_csv('../../2016/PGE/D1977_OFFICE_LBNL.csv',delimiter=',')
# customers.drop(customers.columns[[0]], inplace=True, axis=1)

In [159]:
with open('/Users/ryin/Dropbox/LBNL/PGE/stockton/saids.csv','w') as csvfile:
    writer = csv.writer(csvfile,delimiter=',')
    header = ['SAID','StartDate','EndDate','ZIPCODE','SUBLAP']
    writer.writerow(header)
    
    for said in set(df.SA):
        df2 = df[df['SA'] == said]
        writer.writerow([said,df2.DATE.min().strftime('%Y-%m-%d'),df2.DATE.max().strftime('%Y-%m-%d'), df2.ZIPCODE[0],df2.SUBLAP[0]])

customer_output = '/Users/ryin/Dropbox/LBNL/PGE/stockton/customers_95202.csv'
if os.path.exists(customer_output):
    os.remove(customer_output)
    
with open(customer_output,'a') as csvfile:
    writer = csv.writer(csvfile,delimiter=',')
    writer.writerow(customers.columns)

    for said in set(df.SA):
        df_said = customers[customers['sa_id'] == said]
        df_said.to_csv(csvfile, header=False, index=False)
    print('Customer Output is Completed.')

Customer Output is Completed.


In [160]:
def str2float(x):
    try:
        return("{0:.3f}".format(float(x)))
    except ValueError:
        return np.nan

## Start the analyze the dataset

In [161]:
def CleanMeterData(df, SAID, weather, holidays):
    df2 = df[df['SA'] == SAID].sort_index(axis=0)
    df3 = pd.DataFrame(df2[df2.columns[8:104]].stack(),columns=['energykWh'])
    df3.index.names = ['Date','Time']

    dateindex = []
    for datenum in range(len(df3)):
        dateindex.append(df3.index[datenum][0].strftime('%Y-%m-%d')+' '+df3.index[datenum][1])

    df3.index = pd.to_datetime(dateindex, format='%Y-%m-%d %H:%M')
    
    df3['power'] = df3['energykWh'].apply(lambda x: str2float(x))
    df3[['power']] = df3[['power']].astype(float)
    df3.power = df3.power * 4 # convert kWh to kW

    df3['day'] = list(map(lambda x: x.strftime('%Y/%m/%d'),df3.index))
    df3['time'] = list(map(lambda x: x.strftime('%H:%M'),df3.index))
    df3['month'] = list(map(lambda x: x.strftime('%m'),df3.index))
    df3['hour'] = list(map(lambda x: x.strftime('%H'),df3.index))
    df3['month'] = df3.month.astype(int)
    df3['hour'] = df3.hour.astype(int)
    df3['weekday'] = df3.index.weekday
    df3['holiday'] = list(map(lambda x: x.strftime('%Y-%m-%d') in holidays.day.values,df3.index))
    df3['holiday'] = df3.holiday.astype(int)
    
    data = pd.concat([df3,weather],axis=1,join_axes=[df3.index])
    
    data.fillna(method='pad')

    # calculate basepower on weekdays
    data.loc[:,'basepower'] = 0
    data.loc[:,'peakoat'] = 0

    # calculate the daily peak OAT
    PeakOAT = data.groupby(['day'])['oat'].max()

    for day in PeakOAT.index:
        data.loc[(data['day'] == day),'peakoat'] = PeakOAT[day]

    # subset the weekday and weekend power
    df_wd = data.loc[(data['weekday'] >= 0) & (data['weekday'] <= 4) & (data['holiday'] == 0)]
    df_wk = data.loc[(data.weekday.apply(lambda x: x in [5,6])) | (data['holiday'] == 1)]

    # calculate the basepower on weekdays and weekend
    df_wd_base = df_wd.loc[(df_wd['peakoat'] < 45)]
    df_wd_basedaily = df_wd_base.pivot(index='time',columns='day',values='power')

    for tim in df_wd_basedaily.index:
        data.loc[(data['time'] == tim) & (data['weekday'] >= 0) & \
           (data['weekday'] <= 4) & (data['holiday'] == 0),'basepower'] = df_wd_basedaily.mean(axis=1)[tim]
        data.loc[(data['time'] == tim) & (data.weekday.apply(lambda x: x in [5,6])),'basepower'] = df_wd_basedaily.mean(axis=1)[tim]
        data.loc[(data['time'] == tim) & (data['holiday'] == 1),'basepower'] = df_wd_basedaily.mean(axis=1)[tim]

    data.loc[:,'hvac'] = data.power - data.basepower
    
    return data

In [162]:
# consider the holidays
holidays = pd.read_csv('holiday.csv', usecols=[0])
holidays['date'] = pd.to_datetime(pd.Series(holidays['date']), format='%m/%d/%y')
holidays['day'] = holidays.date.apply(lambda x: x.strftime('%Y-%m-%d'))



In [163]:
weather = pd.read_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/stockon/KCASTOCK9.temperatureinterpolated.csv',header=None)
weather.columns = ['Date','oat']
weather.index = pd.to_datetime(weather.Date, format='%m/%d/%Y %H:%M')
weather = weather.drop(['Date'],axis=1)
weather = weather.groupby(weather.index).first() # remove duplicated Index in DataFrame
weather = weather.asfreq('15min', method='pad') # fill missing timeseries data in DataFrame


In [ ]:
def read_weather(weather_file):
    weather = pd.read_csv(weather_file,header=None)
    weather.columns = ['Date','oat']
    weather.index = pd.to_datetime(weather.Date, format='%m/%d/%Y %H:%M')
    weather = weather.drop(['Date'],axis=1)
    weather = weather.groupby(weather.index).first() # remove duplicated Index in DataFrame
    weather = weather.asfreq('15min', method='pad') # fill missing timeseries data in DataFrame
    return weather

## 2. Generate all the basic metric from meter data and weather data

In [245]:
def calculateCustMetrics(df, customers, SAID):
    
    df_wd = df.loc[(df['weekday'] >= 0) & (df['weekday'] <= 4) & (df['holiday'] == 0)]
    df_cssb = df_wd.loc[(df_wd['month'] >= 5) & (df_wd['month'] <= 10),['power','hour','time','oat']]
    CSSB = df_cssb.groupby(['time'])['power'].mean()

    hotDays = df_wd.groupby(['day'])['oat'].max().sort_values(ascending=False)[0:10]
    hotDaysPower = df.loc[saidData['day'].isin(hotDays.index),['power', 'oat','time','day']]
    hotDaysHVAC = df.loc[saidData['day'].isin(hotDays.index),['hvac', 'oat','time','day']]

    custHVAC = hotDaysHVAC.pivot(index='time',columns='day',values='hvac')
    custHVAC.columns = ['hotDayHVAC0','hotDayHVAC1','hotDayHVAC2','hotDayHVAC3','hotDayHVAC4',
                        'hotDayHVAC5','hotDayHVAC6','hotDayHVAC7','hotDayHVAC8','hotDayHVAC9']
    custHVAC['hotDayHVAC'] = hotDaysHVAC.groupby(['time'])['hvac'].mean()

    custMetrics = hotDaysPower.pivot(index='time',columns='day',values='power')
    custMetrics.columns = ['hotDay0','hotDay1','hotDay2','hotDay3','hotDay4',
                           'hotDay5','hotDay6','hotDay7','hotDay8','hotDay9']
    custMetrics['cssb'] = df_cssb.groupby(['time'])['power'].mean()
    custMetrics['hotDay'] = hotDaysPower.groupby(['time'])['power'].mean()
    custMetrics['peakDay'] = df.loc[df['day'] == df.power.idxmax().strftime('%Y/%m/%d'),'power'].values

    DR_Equations = pd.read_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/DREquations.csv')
    DR_Equations.index = DR_Equations.DR_Controls

    custOAT = hotDaysPower.pivot(index='time',columns='day',values='oat')
    custOAT.columns = ['hotDayOAT0','hotDayOAT1','hotDayOAT2','hotDayOAT3','hotDayOAT4','hotDayOAT5','hotDayOAT6',
                       'hotDayOAT7','hotDayOAT8','hotDayOAT9']
    custOAT['cssb_oat'] = df_cssb.groupby(['time'])['oat'].mean()
    custOAT['hotDay_oat'] = hotDaysPower.groupby(['time'])['oat'].mean()
    custOAT['peakDay_oat'] = df.loc[df['day'] == df.power.idxmax().strftime('%Y/%m/%d'),'oat'].values
    custOAT['hour'] = df.loc[df['day'] == df.power.idxmax().strftime('%Y/%m/%d'),'hour'].values
    custOAT['NoPrecool_Reset2F_alpha'] = np.nan
    custOAT['NoPrecool_Reset2F_beta'] = np.nan
    custOAT['NoPrecool_Reset4F_alpha'] = np.nan
    custOAT['NoPrecool_Reset4F_beta'] = np.nan
    custOAT['NoPrecool_Reset6F_alpha'] = np.nan
    custOAT['NoPrecool_Reset6F_beta'] = np.nan
    custOAT['Precool2F_Reset2F_alpha'] = np.nan
    custOAT['Precool2F_Reset2F_beta'] = np.nan
    custOAT['Precool2F_Reset4F_alpha'] = np.nan
    custOAT['Precool2F_Reset4F_beta'] = np.nan
    custOAT['Precool2F_Reset6F_alpha'] = np.nan
    custOAT['Precool2F_Reset6F_beta'] = np.nan

    for hour in range(14,18):
        custOAT.loc[custOAT['hour'] == hour, ['NoPrecool_Reset2F_alpha','NoPrecool_Reset2F_beta']] = \
            DR_Equations.loc[(DR_Equations['hour'] == hour) & (DR_Equations['precool'] == 0) & (DR_Equations['reset'] == 2), ['alpha_2','beta_2']].values
        custOAT.loc[custOAT['hour'] == hour, ['NoPrecool_Reset4F_alpha','NoPrecool_Reset4F_beta']] = \
            DR_Equations.loc[(DR_Equations['hour'] == hour) & (DR_Equations['precool'] == 0) & (DR_Equations['reset'] == 4), ['alpha_2','beta_2']].values
        custOAT.loc[custOAT['hour'] == hour, ['NoPrecool_Reset6F_alpha','NoPrecool_Reset6F_beta']] = \
            DR_Equations.loc[(DR_Equations['hour'] == hour) & (DR_Equations['precool'] == 0) & (DR_Equations['reset'] == 6), ['alpha_2','beta_2']].values

        custOAT.loc[custOAT['hour'] == hour, ['Precool2F_Reset2F_alpha','Precool2F_Reset2F_beta']] = \
            DR_Equations.loc[(DR_Equations['hour'] == hour) & (DR_Equations['precool'] == 2) & (DR_Equations['reset'] == 2), ['alpha_2','beta_2']].values
        custOAT.loc[custOAT['hour'] == hour, ['Precool2F_Reset4F_alpha','Precool2F_Reset4F_beta']] = \
            DR_Equations.loc[(DR_Equations['hour'] == hour) & (DR_Equations['precool'] == 2) & (DR_Equations['reset'] == 4), ['alpha_2','beta_2']].values
        custOAT.loc[custOAT['hour'] == hour, ['Precool2F_Reset6F_alpha','Precool2F_Reset6F_beta']] = \
            DR_Equations.loc[(DR_Equations['hour'] == hour) & (DR_Equations['precool'] == 2) & (DR_Equations['reset'] == 6), ['alpha_2','beta_2']].values

    result = pd.concat([custMetrics, custOAT, custHVAC], axis=1)

    usrInput = customers[customers['sa_id'] == SAID]
    usrOutput = pd.DataFrame()

    usrOutput['SAID'] = usrInput.sa_id
    usrOutput['UUID'] = uuid.uuid4()
    usrOutput.index = usrOutput['UUID']
    usrOutput['Peak_kW'] = df.power.max()
    usrOutput['Peak_kW_timestamp'] = df.power.idxmax()
    if df.power.max() >=200:
        usrOutput['CustSize'] = 'Large'
    else:
        usrOutput['CustSize'] = 'Small/Medium'
    usrOutput['NAICS'] = usrInput.sa_naics_cd.values
    usrOutput['NAICS_desc'] = usrInput.naics_desc.values
    usrOutput['cty'] = usrInput.cty_nm.values
    usrOutput['ZIPCODE'] = usrInput.zip_5_dgt.values
    usrOutput['SUBLAP'] = 'SLAP_PGSN-APND'
    usrOutput['PGE_CZ'] = 5
    usrOutput['CEC_CZ'] = 3
    usrOutput['AMP'] = usrInput.AMP.values
    usrOutput['PDP'] = usrInput.PDP.values
    usrOutput['DBP'] = usrInput.DBP.values
    usrOutput['CBP'] = usrInput.CBP.values
    usrOutput['PDP_TOU_Ratio'] = 1.0

    usrOutput['DR_Capacity_Precool0F_Reset2F'] = np.mean(result.peakDay * (result.NoPrecool_Reset2F_beta + result.peakDay_oat * result.NoPrecool_Reset2F_alpha)/100)
    usrOutput['DR_Capacity_Precool0F_Reset4F'] = np.mean(result.peakDay * (result.NoPrecool_Reset4F_beta + result.peakDay_oat * result.NoPrecool_Reset4F_alpha)/100)
    usrOutput['DR_Capacity_Precool0F_Reset6F'] = np.mean(result.peakDay * (result.NoPrecool_Reset6F_beta + result.peakDay_oat * result.NoPrecool_Reset6F_alpha)/100)
    usrOutput['DR_Capacity_Precool2F_Reset2F'] = np.mean(result.peakDay * (result.Precool2F_Reset2F_beta + result.peakDay_oat * result.Precool2F_Reset2F_alpha)/100)
    usrOutput['DR_Capacity_Precool2F_Reset4F'] = np.mean(result.peakDay * (result.Precool2F_Reset4F_beta + result.peakDay_oat * result.Precool2F_Reset4F_alpha)/100)
    usrOutput['DR_Capacity_Precool2F_Reset6F'] = np.mean(result.peakDay * (result.Precool2F_Reset6F_beta + result.peakDay_oat * result.Precool2F_Reset6F_alpha)/100)

    usrOutput['DR_Capacity_CycleOnOff_30Pct'] = np.mean(result['14:00':'17:45'].hotDayHVAC) * 0.3
    usrOutput['DR_Capacity_CycleOnOff_50Pct'] = np.mean(result['14:00':'17:45'].hotDayHVAC) * 0.5
    usrOutput['DR_Capacity_CycleOnOff_100Pct'] = np.mean(result['14:00':'17:45'].hotDayHVAC) * 1.0
    usrOutput['HVAC_Building_Ratio'] = usrOutput['DR_Capacity_CycleOnOff_100Pct'] / usrOutput['Peak_kW']

#     usrOutput.to_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/CustSummary.csv')
    return result, usrOutput

In [226]:
ZipCodeGroup = pd.read_csv('/Users/ryin/Dropbox/LBNL/PGE/ADRGEOOfficeZips.csv')



In [246]:
for zc in ZipCodeGroup[ZipCodeGroup['City']=='Stockton']['Zip5'][0:7]:
    df = pd.read_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/'+str(zc)+'.csv',header=None)
    df.drop(df.columns[[0]], inplace=True, axis=1)
    df.columns = header

    df.DATE = pd.to_datetime(df.DATE, format='%m/%d/%Y')
    df.index = pd.to_datetime(df.DATE, format='%m/%d/%Y')
    
    with open('/Users/ryin/Dropbox/LBNL/PGE/stockton/CustSummary_'+str(zc)+'.csv','w') as csvfile:
        writer = csv.writer(csvfile,delimiter=',')
        csvheader = ['UUID','SAID','UUID','Peak_kW','Peak_kW_timestamp','CustSize','NAICS',
                  'NAICS_desc','cty','ZIPCODE','SUBLAP','PGE_CZ','CEC_CZ','AMP','PDP','DBP','CBP','PDP_TOU_Ratio',
                  'DR_Capacity_Precool0F_Reset2F','DR_Capacity_Precool0F_Reset4F','DR_Capacity_Precool0F_Reset6F',
                  'DR_Capacity_Precool2F_Reset2F','DR_Capacity_Precool2F_Reset4F','DR_Capacity_Precool2F_Reset6F',
                  'DR_Capacity_CycleOnOff_30Pct','DR_Capacity_CycleOnOff_50Pct','DR_Capacity_CycleOnOff_100Pct',
                  'HVAC_Building_Ratio']
        writer.writerow(csvheader)

    for SAID in set(df.SA):
            saidData = CleanMeterData(df, SAID, weather, holidays)
            metrics = pd.DataFrame()
            custSummary = pd.DataFrame()
            metrics, custSummary = calculateCustMetrics(saidData, customers, SAID)
            metrics.to_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/CustMetrics_'+str(zc)+'.csv',mode='w',sep=',',header=True)
            custSummary.to_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/CustSummary_'+str(zc)+'.csv',mode='a',sep=',',header=False)
            print('Customer ' + str(SAID)+' is Completed.')

/Users/ryin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Customer 838288097.0 is Completed.
Customer 1605680066.0 is Completed.
Customer 2041448326.0 is Completed.
Customer 398583910.0 is Completed.
Customer 1152913159.0 is Completed.
Customer 2051150602.0 is Completed.
Customer 1659186381.0 is Completed.
Customer 1556177005.0 is Completed.
Customer 1670765005.0 is Completed.
Customer 891901840.0 is Completed.
Customer 1627410353.0 is Completed.
Customer 48035508.0 is Completed.
Customer 1738264661.0 is Completed.
Customer 1502410805.0 is Completed.
Customer 729786169.0 is Completed.
Customer 323154585.0 is Completed.
Customer 1868800378.0 is Completed.
Customer 1856089211.0 is Completed.
Customer 60120025.0 is Completed.
Customer 1462405501.0 is Completed.
Customer 2117937756.0 is Completed.


/Users/ryin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,18,19,20,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Customer 1150280962.0 is Completed.
Customer 909608194.0 is Completed.
Customer 183622789.0 is Completed.
Customer 1005911942.0 is Completed.
Customer 1615063816.0 is Completed.
Customer 183622930.0 is Completed.
Customer 1830868884.0 is Completed.
Customer 1569725590.0 is Completed.
Customer 1038915097.0 is Completed.
Customer 745972122.0 is Completed.
Customer 413915162.0 is Completed.
Customer 425217053.0 is Completed.
Customer 398583838.0 is Completed.
Customer 255919272.0 is Completed.
Customer 1090486702.0 is Completed.
Customer 1615063348.0 is Completed.
Customer 530726454.0 is Completed.
Customer 1909546046.0 is Completed.
Customer 1497249474.0 is Completed.
Customer 2100573252.0 is Completed.
Customer 148083655.0 is Completed.
Customer 438847564.0 is Completed.
Customer 1454156796.0 is Completed.
Customer 175649998.0 is Completed.
Customer 1421660879.0 is Completed.
Customer 1220991697.0 is Completed.
Customer 245972433.0 is Completed.
Customer 1171340121.0 is Completed.
Custo

/Users/ryin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Customer 1477629856.0 is Completed.
Customer 1286689473.0 is Completed.
Customer 1286689442.0 is Completed.
Customer 829741667.0 is Completed.
Customer 955364068.0 is Completed.
Customer 163074021.0 is Completed.
Customer 955364934.0 is Completed.
Customer 1286689835.0 is Completed.
Customer 806241069.0 is Completed.
Customer 1308595567.0 is Completed.
Customer 1456014353.0 is Completed.
Customer 1563597201.0 is Completed.
Customer 1789347987.0 is Completed.
Customer 1286689364.0 is Completed.
Customer 1563597077.0 is Completed.
Customer 1136224503.0 is Completed.
Customer 1286689177.0 is Completed.
Customer 748254972.0 is Completed.
Customer 382092800.0 is Completed.
Customer 1209904261.0 is Completed.
Customer 1325393926.0 is Completed.
Customer 1867064454.0 is Completed.
Customer 1662609545.0 is Completed.
Customer 1659089290.0 is Completed.
Customer 1075394697.0 is Completed.
Customer 777085321.0 is Completed.
Customer 1087920531.0 is Completed.
Customer 1320898452.0 is Completed.


/Users/ryin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Customer 780401161.0 is Completed.
Customer 310174733.0 is Completed.
Customer 312860440.0 is Completed.
Customer 2121717786.0 is Completed.
Customer 863735323.0 is Completed.
Customer 742506403.0 is Completed.
Customer 2121717541.0 is Completed.
Customer 310174513.0 is Completed.
Customer 1925164594.0 is Completed.
Customer 659172529.0 is Completed.
Customer 2121717814.0 is Completed.
Customer 1757254201.0 is Completed.
Customer 335105602.0 is Completed.
Customer 730948036.0 is Completed.
Customer 1479993036.0 is Completed.
Customer 1639939279.0 is Completed.
Customer 335105365.0 is Completed.
Customer 398583398.0 is Completed.
Customer 1993744872.0 is Completed.
Customer 2121717866.0 is Completed.
Customer 12192243.0 is Completed.
Customer 1297513843.0 is Completed.
Customer 12192502.0 is Completed.
Customer 1532324601.0 is Completed.


In [ ]:
def demandresponse_results(city):
    for zc in ZipCodeGroup[ZipCodeGroup['City']==city]['Zip5']:
        df = pd.read_csv('/home/ryin/Documents/pge-data/D1977ZipCode/'+str(zc)+'.csv',header=None)
        df.drop(df.columns[[0]], inplace=True, axis=1)
        df.columns = header

        df.DATE = pd.to_datetime(df.DATE, format='%m/%d/%Y')
        df.index = pd.to_datetime(df.DATE, format='%m/%d/%Y')

        with open('/home/ryin/Documents/pge-data/results/'+city+'/CustSummary_'+str(zc)+'.csv','w') as csvfile:
            writer = csv.writer(csvfile,delimiter=',')
            csvheader = ['UUID','SAID','UUID','Peak_kW','Peak_kW_timestamp','CustSize','NAICS',
                      'NAICS_desc','cty','ZIPCODE','SUBLAP','PGE_CZ','CEC_CZ','AMP','PDP','DBP','CBP','PDP_TOU_Ratio',
                      'DR_Capacity_Precool0F_Reset2F','DR_Capacity_Precool0F_Reset4F','DR_Capacity_Precool0F_Reset6F',
                      'DR_Capacity_Precool2F_Reset2F','DR_Capacity_Precool2F_Reset4F','DR_Capacity_Precool2F_Reset6F',
                      'DR_Capacity_CycleOnOff_30Pct','DR_Capacity_CycleOnOff_50Pct','DR_Capacity_CycleOnOff_100Pct',
                      'HVAC_Building_Ratio']
            writer.writerow(csvheader)

        for SAID in set(df.SA):
                saidData = CleanMeterData(df, SAID, weather, holidays)
                metrics = pd.DataFrame()
                custSummary = pd.DataFrame()
                metrics, custSummary = calculateCustMetrics(saidData, customers, SAID)
                metrics.to_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/CustMetrics_'+str(zc)+'.csv',mode='w',sep=',',header=True)
                custSummary.to_csv('/Users/ryin/Dropbox/LBNL/PGE/stockton/CustSummary_'+str(zc)+'.csv',mode='a',sep=',',header=False)
#                 print('Customer ' + str(SAID)+' is Completed.')
        print('DR Analytics for Customers in ' + city +' '+ str(zc)+' are Completed.')